In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [2]:
def load_data(directory):
    data = []
    vehicle_ids = []

    for i, file_name in enumerate(os.listdir(directory)):

        if(i % 5000 == 0):
            print(f'Loading {i}')
            
        # Extract vehicle IDs from the filename
        vehicle_id = int(file_name.split('_')[0])
        vehicle_ids.append(vehicle_id)

        # Load NPY files
        file_path = os.path.join(directory, file_name)
        loaded_data = np.load(file_path)

        data.append(loaded_data)
        
        # Reshape loaded data and append to a list
        #reshaped_data = loaded_data.reshape((1, 32, 64))
        # data.append(reshaped_data)

    # Converting data and labels to PyTorch Tensors
    data = np.array(data)
    data = torch.from_numpy(data)
    vehicle_ids = torch.tensor(vehicle_ids, dtype=torch.long)
    
    # Adjust labels to be in range 0 to 1361
    if(torch.min(vehicle_ids).item() == 1):
        vehicle_ids = vehicle_ids - 1
    
    return (data, vehicle_ids)

In [3]:
# Load train data and labels from train directory
train_directory = "vehicle-x/train"
test_directory = "vehicle-x/test"

train_data, vehicle_ids_train = load_data(train_directory)
test_data, vehicle_ids_test = load_data(test_directory)

Loading 0
Loading 5000
Loading 10000
Loading 15000
Loading 20000
Loading 25000
Loading 30000
Loading 35000
Loading 40000
Loading 45000
Loading 0
Loading 5000
Loading 10000
Loading 15000


In [4]:
# Hyperparameters
num_epochs = 100
batch_size = 32
test_batch_size = 64
learning_rate = 0.01

class CustomDataset():
    def __init__(self, data_array, labels):
        self.data = data_array
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Create custom dataset from the tensors
train_dataset = CustomDataset(train_data, vehicle_ids_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = CustomDataset(test_data, vehicle_ids_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size, shuffle=True)

In [5]:
# Define the constructive cascade network
class CascadeLayer(nn.Module):
    def __init__(self, input_size, output_size):
        super(CascadeLayer, self).__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        return torch.tanh(self.fc(x))

class CascadeNN(nn.Module):
    def __init__(self, input_size, output_size, initial_hidden_size=64):
        super(CascadeNN, self).__init__()
        self.cascade_hidden_size = 32
        self.input_size = input_size
        self.output_size = output_size
        self.layers = nn.ModuleList()

        # Initial layer
        self.layers.append(CascadeLayer(input_size, initial_hidden_size))

        # Output layer
        self.output_layer = nn.Linear(initial_hidden_size + self.input_size, output_size)

    def add_cascade_layer(self):
        # The new layer will take input from the last cascade layer and the original input
        new_input_size = self.layers[-1].fc.out_features + self.layers[-1].fc.in_features
        new_layer = CascadeLayer(new_input_size, self.cascade_hidden_size).to(device)
        self.layers.append(new_layer)

        # Adjust the output layer to accomodate the new cascade layer
        self.output_layer = nn.Linear(self.output_layer.in_features + self.cascade_hidden_size, self.output_size).to(device)

    
    def forward(self, x):
        outputs = [x]

        for layer in self.layers:
            combined_input = torch.cat(outputs, dim=1)
            layer_output = layer(combined_input)
            outputs.append(layer_output)
        
        final_combined_input = torch.cat(outputs, dim=1)
        logits = self.output_layer(final_combined_input)
        return nn.functional.log_softmax(logits, dim=1)


# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialise network
model = CascadeNN(2048, 1362).to(device)

print(model)


CascadeNN(
  (layers): ModuleList(
    (0): CascadeLayer(
      (fc): Linear(in_features=2048, out_features=64, bias=True)
    )
  )
  (output_layer): Linear(in_features=2112, out_features=1362, bias=True)
)


In [6]:
def train():
    model.train()
    correct = 0
    for batch_idx, (batch_data, batch_labels) in enumerate(train_loader):
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_data)

        pred = outputs.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(batch_labels.data.view_as(pred)).long().cpu().sum()
        
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], step[{batch_idx + 1}], loss: {loss.item():.4f}')

    accuracy = correct / len(train_loader.dataset)
    print(f'Train Accuracy {correct}/{len(train_loader.dataset)} ({accuracy:.4f})')
    return accuracy

In [7]:
def test():
    model.eval()
    correct = 0

    for batch_data, batch_labels in test_loader:
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        
        outputs = model(batch_data)
        
        pred = outputs.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(batch_labels.data.view_as(pred)).long().cpu().sum()

    accuracy = correct / len(test_loader.dataset)
    print(f'Test Accuracy {correct}/{len(train_loader.dataset)} ({accuracy:.4f})')
    return accuracy

In [8]:
# Train the model
criterion = nn.NLLLoss()
#optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
optimizer = optim.Rprop(model.parameters())

best_accuracy = 0.0
epochs_without_improvement = 0
patience = 5

for epoch in range(num_epochs):
    train_accuracy = train()
    test_accuracy = test()

    if(train_accuracy > best_accuracy):
        best_accuracy = train_accuracy
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1

    # If performance hasn't improved for 'patience' epochs, add a layer
    if(epochs_without_improvement == patience):
        print("Performance hasn't improved for {} epochs. Adding a new cascade layer.".format(patience))
        model.add_cascade_layer()
        epochs_without_improvement = 0

Epoch [1/100], step[100], loss: 82.7862
Epoch [1/100], step[200], loss: 87.4881
Epoch [1/100], step[300], loss: 84.5644
Epoch [1/100], step[400], loss: 85.9298
Epoch [1/100], step[500], loss: 102.7145
Epoch [1/100], step[600], loss: 100.9594
Epoch [1/100], step[700], loss: 106.6409
Epoch [1/100], step[800], loss: 90.3357
Epoch [1/100], step[900], loss: 111.8907
Epoch [1/100], step[1000], loss: 120.7866
Epoch [1/100], step[1100], loss: 102.1017
Epoch [1/100], step[1200], loss: 132.3623
Epoch [1/100], step[1300], loss: 139.6849
Epoch [1/100], step[1400], loss: 152.3833
Train Accuracy 39/45438 (0.0009)
Test Accuracy 34/45438 (0.0022)
Epoch [2/100], step[100], loss: 145.5549
Epoch [2/100], step[200], loss: 159.4249
Epoch [2/100], step[300], loss: 157.5732
Epoch [2/100], step[400], loss: 179.4848
Epoch [2/100], step[500], loss: 198.4363
Epoch [2/100], step[600], loss: 183.9865
Epoch [2/100], step[700], loss: 222.1593
Epoch [2/100], step[800], loss: 230.7349
Epoch [2/100], step[900], loss: 2

KeyboardInterrupt: 